In [1]:
import cv2
import os
import numpy as np
from random import shuffle
from tqdm import tqdm

TRAIN_DIR = "c:/Users/Rohit/Downloads/ML/Neural Networks/Cats & Dogs"
IMG_SIZE = 50

In [2]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE),(IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy',training_data)
    return training_data

In [3]:
training_data = np.load('train_data.npy',allow_pickle=True)

In [22]:
import torch

In [23]:
import torch.nn as nn
import torch.nn.functional as F

In [24]:
class Net(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,32,5)
        self.conv2 = nn.Conv2d(32,64,5)
        self.conv3 = nn.Conv2d(64,128,5)
        
        x = torch.randn(50,50).view(-1,1,50,50)
        self._to_linear = None
        self.convs(x)
        
        self.fc1 = nn.Linear(self._to_linear,512)
        self.fc2 = nn.Linear(512,2)
        
    def convs(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv3(x)),(2,2))
        
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x
    
    def forward(self,x):
        x = self.convs(x)
        x = x.view(-1,self._to_linear)
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x),dim=1)
        return x
 
net = Net()

In [44]:
import torch.optim as optim

    optimizer = optim.Adam(net.parameters(),lr=0.001)
    loss_fuction = nn.MSELoss()

x = torch.Tensor([i[0] for i in training_data]).view(-1,50,50)
x = x/255
y = torch.Tensor([i[1] for i in training_data])

VAL_PCT = 0.1
val_size = int((len(x)*VAL_PCT))

train_X= x[val_size:]
test_X = x[:val_size]

train_y = y[val_size:]
test_y = y[:val_size]

2500


In [50]:
# def train():
    
#     BATCH_SIZE = 100
#     EPOCHS = 3

#     for epoch in range(EPOCHS):
#         for i in tqdm(range(0,len(train_X),BATCH_SIZE)):
#             batch_X = train_X[i:i+BATCH_SIZE].view(-1,1,50,50)
#             batch_y = train_y[i:i+BATCH_SIZE]
        
#             net.zero_grad()
#             outputs = net(batch_X)
#             loss = loss_fuction(outputs,batch_y)
#             loss.backward()
#             optimizer.step()
        
#         print(loss)

100%|████████████████████████████████████████████████████████████████████████████████| 225/225 [02:59<00:00,  1.26it/s]

tensor(0.1200, grad_fn=<MseLossBackward>)


In [51]:
# def test():
    
#     correct = 0
#     total = 0

#     with torch.no_grad():
#         for i in tqdm(range(len(test_X))):
#             real_class = torch.argmax(test_y[i])
#             net_out = net(test_X[i].view(-1,1,50,50))[0]
#             predicted_class = torch.argmax(net_out) 
#             if predicted_class == real_class:
#                 correct += 1
#             total +=1
#     print('Accuracy',round(correct/total,3))     

100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:15<00:00, 161.55it/s]

Accuracy 0.777


In [52]:
def fwd_pass(X,y,train=False):
    if train:
        net.zero_grad()
    outputs = net(X)
    matches = [torch.argmax(i) == torch.argmax(j) for i,j in zip(outputs,y)]
    acc = matches.count(True)/len(matches)
    loss = loss_fuction(outputs,y)
    
    if train:
        loss.backward()
        optimizer.step()
        
    return acc , loss    

In [54]:
def test(size=32):
    
    random_start = np.random.randint(len(test_X)-size)
    X,y = test_X[random_start:random_start+size],test_y[random_start:random_start+size]
    with torch.no_grad():
        val_acc,val_loss = fwd_pass(X.view(-1,1,50,50),y)
    return val_acc , val_loss

val_acc,val_loss = test(size=32)
print(val_acc,val_loss)
    

0.875 tensor(0.1391)


In [56]:
import time

MODEL_NAME = f'model_3CNN_32_64_128_2FC_512_2 {int(time.time())}'

net = Net()

optimizer = optim.Adam(net.parameters(),lr=0.001)
loss_fuction = nn.MSELoss()

print(MODEL_NAME)

def train():
    
    BATCH_SIZE = 100
    EPOCHS = 1
    with  open('model.log','a') as f:
        for epoch in range(EPOCHS):
            for i in tqdm(range(0,len(train_X),BATCH_SIZE)):
                batch_X = train_X[i:i+BATCH_SIZE].view(-1,1,50,50)
                batch_y = train_y[i:i+BATCH_SIZE]
                
                acc , loss = fwd_pass(batch_X, batch_y , train = True)
                if i % 50 == 0:
                    val_acc , val_loss = test(size = 100)
                    f.write(f'{MODEL_NAME},{round(time.time(),3)},{round(float(acc),2)},{round(float(loss),4)},{round(float(val_acc),2)},{round(float(val_loss),4)}\n')
    
train()



  0%|                                                                                          | 0/225 [00:00<?, ?it/s]

model_3CNN_32_64_128_2FC_512_2 1585942236


100%|████████████████████████████████████████████████████████████████████████████████| 225/225 [04:25<00:00,  1.18s/it]
